In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from skimage.io import imread

from chromemt_analysis import segment_like_paper, continuous_erosion, linear_fit_to_residual_volume 

In [ ]:
# fig. 4 EMT volume
img = imread('/Volumes/davidh-ssd/chromemt_data/49801.tif')

# rough location of first patch
cut_siz = 94 # 94px ~ 120nm at 1.28nm pixel size
off = 0, 140, 40

cut = img[tuple((slice(o, o+cut_siz) for o in off))]

In [ ]:
# test linear fit
rv_test = np.concatenate([np.linspace(1,0.5,5),np.linspace(0.5, 0.1, 15)] ) + np.random.rayleigh(scale=0.1, size=20)
r, mod = linear_fit_to_residual_volume(rv_test)

# plot trace, fitted model line
plt.plot(rv_test)
plt.plot(mod.predict(np.linspace(1,0).reshape(-1,1))-1, np.linspace(1,0).reshape(-1,1))

In [ ]:
mask = segment_like_paper(cut)
erosion_radius=np.arange(1, 15)
residual_vol = continuous_erosion(mask, erosion_radius)

In [ ]:
d, mod = linear_fit_to_residual_volume(residual_vol, erosion_radius)

# plot residual volume fraction
plt.plot(erosion_radius, residual_vol)
# plot fitted line
plt.plot(mod.predict(np.linspace(residual_vol[0],0).reshape(-1,1)), np.linspace(residual_vol[0],0).reshape(-1,1))

# diameter in nm
d * 1.28

In [ ]:
fig, axs = plt.subplots(ncols=2)
axs[0].imshow(cut[0])
axs[1].imshow(segment_like_paper(cut)[0])

# chromatin volume fraction
segment_like_paper(cut).sum() / np.prod(cut.shape)

In [ ]:
def hypersphere(rank, radius):
    '''
    rank-n hypersphere, should be just like disk/ball
    '''
    return np.linalg.norm(np.stack(np.meshgrid(*[np.arange(-radius, radius+1)]*rank, indexing='ij')), axis=0)<=radius

patch_eq = equalize_adapthist(cut, 78)
mask = patch_eq < threshold_li(patch_eq)

fig, axs = plt.subplots(ncols=3, figsize=(10,5))
axs[0].imshow(patch_eq[0])
axs[1].imshow(mask[0])
axs[2].imshow(median_filter(mask[0], footprint=disk(2.5)))